In [209]:
def cum_weight(seq):
    val = 0.0
    for att in seq:
        val = val + att_sensitivity[att]
    return val/len(seq)

In [198]:
class SeqTuple(object):
    def __init__(self, seq, tsmw):
        self.seq = seq
        self.cnt = 0
        self.swub = 0
        self.wsup = 0
        self.weight = cum_weight(seq)/tsmw

In [162]:
def get_loc(pattern, seq):
    indices = []
    for ind in range(len(seq)):
        if seq[ind]==pattern[0]:
            flag = True
            for i in range(len(pattern)):
                if ind+i>=len(seq) or seq[ind+i]!=pattern[i]:
                    flag = False
                    break
            if flag==True:
                indices.append(ind)
    return indices

In [218]:
def finding_WS(x, sdbi, r, thresh, sensitivity, tsmw):
    
    smw = {}
    # tid is transaction id
    for tid in sdbi:
        seq = sdbi[tid]
        smw_seq = 0
        for att in seq:
            smw_seq = max(smw_seq, sensitivity[att])
        smw[tid] = smw_seq
    
    TS = {}
    for tid in sdbi:
        seq = sdbi[tid]
        ind = get_loc(x, seq)
        #print(ind,x,seq)
        for i in range(ind[0]+r,len(seq)):
            subseq = list(x)+[seq[i]]
            if subseq not in TS.keys():
                ST = SeqTuple(subseq, tsmw)
                TS[tuple(subseq)] = ST
    
    for seqx in TS:
        for tid in sdbi:
            seq = sdbi[tid]
            ind = get_loc(seqx[-1], seq[r:])
            if len(ind)>0:
                seqx = tuple(seqx)
                TS[seqx].swub += smw[tid]/tsmw
                TS[seqx].cnt += 1
        TS[seqx].wsup = TS[seqx].weight*TS[seqx].cnt
    
    WFUBX = []
    WSX = []
    for pattern in TS.keys():
        print(pattern,TS[pattern].swub,TS[pattern].wsup)
        if TS[pattern].swub >= thresh:
            WFUBX.append(pattern)
        if TS[pattern].wsup >= thresh:
            WSX.append(pattern)
    
    PIX = []
    for ix in WFUBX:
        PIX = PIX + list(ix)
    PIX = set(PIX)
    r = r+1
    
    for tid in sdbi.keys():
        seq = sdbi[tid]
        for att in seq:
            if att not in PIX:
                seq.remove(att)
        if len(seq) < r+1:
            del sdbi[tid]
    
    WSX = []
    WFUBX.sort()
    #print("hi",len(WFUBX))
    for seqx in WFUBX:
        sdbx = {}
        for tid in sdbi:
            seq = sdbi[tid]
            ind = get_loc(seqx[-1], seq[r-1:])
            if len(ind)>0:
                sdbx[len(sdbx.keys())] = list(seqx)+seq[ind[0]+r:]
        print(seqx,sdbx,r)
        WSS = finding_WS(seqx, sdbx, r, thresh, sensitivity, tsmw)
        if len(WSS)==0 and TS[seqx].wsup >= thresh:
            WSS = WSS + [seqx]
        WSX = WSX + WSS
    return WSX

In [222]:
def IWSPAN(sensitivity, sequences, thresh):
    
    smw = {}
    # tid is transaction id
    for tid in sequences:
        seq = sequences[tid]
        #print(seq)
        smw_seq = 0
        for att in seq:
            smw_seq = max(smw_seq, sensitivity[att])
        #print(smw_seq)
        smw[tid] = smw_seq
        
    tsmw = 0.0
    for tid in smw:
        tsmw = tsmw + smw[tid]
    #print(tsmw)
    
    swub = {}
    wsup = {}
    for tid in sequences:
        seq = sequences[tid]
        for att in seq:
            if att in swub:
                if tid not in swub[att]:
                    swub[att].append(tid)
            else:
                swub[att] = [tid]
    for att in swub:
        in_sequences = swub[att]
        wsup[att] = (sensitivity[att]*len(in_sequences))/tsmw
        val = 0
        for tid in in_sequences:
            val = val + smw[tid]
        swub[att] = (val/tsmw)
        #print(att,swub[att],wsup[att])
        
        
    WFUB = []
    WS = []
    attributes = sensitivity.keys() 
    ## attributes represent length 1 subsequences
    for att in attributes:
        if swub[att] >= thresh :
            WFUB.append(att)
        if wsup[att] >= thresh :
            WS.append(att)
    
    r = 1
    PI = WFUB[:]
    for tid in sequences.keys():
        seq = sequences[tid]
        for att in seq:
            if att not in PI:
                seq.remove(att)
        if len(seq) < r+1:
            del sequences[tid]
     
    WSA = []
    WFUB.sort()
    for att in WFUB:
        sdbi = {}
        for tid in sequences:
            seq = sequences[tid]
            ind = get_loc(att, seq)
            if len(ind)>0:
                sdbi[tid] = seq[ind[0]:]
        print(att,sdbi,r)
        WSI = finding_WS([att], sdbi, r, thresh, sensitivity, tsmw)
        if len(WSI)==0 and wsup[att] >= thresh:
            WSI = WSI + [att]
        WSA = WSA + WSI
    return WSA

In [223]:
att_sensitivity = dict({'A':0.1,'B':0.15,'C':0.2,'D':0.3,'E':0.45,'F':0.55,'G':0.65,'H':0.95})
transaction_list = dict({'TID_1': ['B', 'C', 'B'], 'TID_2': ['D','E','C','H','F'], 'TID_3': ['A','C','F','D','E','F'], 'TID_4':['F','G','H'], 'TID_5':['C','D','A','C','E','F']})
print(IWSPAN(att_sensitivity, transaction_list, 0.3))

('A', {'TID_5': ['A', 'C', 'E', 'F'], 'TID_3': ['A', 'C', 'F', 'D', 'E', 'F']}, 1)
(('A', 'D'), 0.171875, 0.0625)
(('A', 'C'), 0.34375, 0.09375000000000001)
(('A', 'F'), 0.34375, 0.203125)
(('A', 'E'), 0.34375, 0.171875)
(('A', 'C'), {0: ['A', 'C', 'E', 'F'], 1: ['A', 'C', 'F', 'E', 'F']}, 2)
(('A', 'C', 'F'), 0.34375, 0.17708333333333334)
(('A', 'C', 'E'), 0.34375, 0.15625)
(('A', 'C', 'E'), {0: ['A', 'C', 'E', 'F'], 1: ['A', 'C', 'E', 'F']}, 3)
(('A', 'C', 'E', 'F'), 0.34375, 0.203125)
(('A', 'C', 'E', 'F'), {}, 4)
(('A', 'C', 'F'), {0: ['A', 'C', 'F'], 1: ['A', 'C', 'F', 'E', 'F']}, 3)
(('A', 'C', 'F', 'F'), 0.171875, 0.109375)
(('A', 'C', 'F', 'E'), 0.171875, 0.1015625)
(('A', 'E'), {0: ['A', 'E', 'F'], 1: ['A', 'E', 'F']}, 2)
(('A', 'E', 'F'), 0.34375, 0.22916666666666669)
(('A', 'E', 'F'), {}, 3)
(('A', 'F'), {0: ['A', 'F'], 1: ['A', 'F', 'E', 'F']}, 2)
(('A', 'F', 'E'), 0.171875, 0.11458333333333334)
(('A', 'F', 'F'), 0.171875, 0.12500000000000003)
('C', {'TID_5': ['C', 'D', 'A'